# Analysis of the Bike Sharing dataset

## About the dataset
The dataset under analysis belongs to bike sharing company based in Chicago and was found on Kaggle at this ([link](https://www.kaggle.com/datasets/karlachang/bike-share-divvy-datasets-from-nov2022-to-oct-2023/data)). The webpage contains a reference to a more complete ([source](https://divvy-tripdata.s3.amazonaws.com/index.html)), where data updated to November 2023 about rides performed by users of the bike-sharing system is available, divided by month.

Here we focus the anlysis on the last quarter of 2022, with the purpose of generating interesting insights on the operations registered in the last part of the year.

## Structure of the notebook
The first part of the notebook is about performing data exploration and cleaning.

The second portion, instead, is dedicated to querying the dataset using MongoDB to get some more interesting insights about rides. The results returned by the queries could be used as support tool for decision making inside the company.

# Import libraries

In [1]:
import pandas as pd
import numpy as np
import random

# Set random seed for reproducibility
random.seed(25)

# Import and inspect data sources


In [2]:
# Import data from CSV files
oct2022 = pd.read_csv('data/202210-divvy-tripdata.csv')
nov2022 = pd.read_csv('data/202211-divvy-tripdata.csv')
dec2022 = pd.read_csv('data/202212-divvy-tripdata.csv')

In [3]:
# Display details of the October 2022 dataset
oct2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558685 entries, 0 to 558684
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             558685 non-null  object 
 1   rideable_type       558685 non-null  object 
 2   started_at          558685 non-null  object 
 3   ended_at            558685 non-null  object 
 4   start_station_name  467330 non-null  object 
 5   start_station_id    467330 non-null  object 
 6   end_station_name    462068 non-null  object 
 7   end_station_id      462068 non-null  object 
 8   start_lat           558685 non-null  float64
 9   start_lng           558685 non-null  float64
 10  end_lat             558210 non-null  float64
 11  end_lng             558210 non-null  float64
 12  member_casual       558685 non-null  object 
dtypes: float64(4), object(9)
memory usage: 55.4+ MB


In [4]:
# Display details of the November 2022 dataset
nov2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337735 entries, 0 to 337734
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             337735 non-null  object 
 1   rideable_type       337735 non-null  object 
 2   started_at          337735 non-null  object 
 3   ended_at            337735 non-null  object 
 4   start_station_name  285778 non-null  object 
 5   start_station_id    285778 non-null  object 
 6   end_station_name    283476 non-null  object 
 7   end_station_id      283476 non-null  object 
 8   start_lat           337735 non-null  float64
 9   start_lng           337735 non-null  float64
 10  end_lat             337505 non-null  float64
 11  end_lng             337505 non-null  float64
 12  member_casual       337735 non-null  object 
dtypes: float64(4), object(9)
memory usage: 33.5+ MB


In [5]:
# Display details of the December 2022 dataset
dec2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181806 entries, 0 to 181805
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             181806 non-null  object 
 1   rideable_type       181806 non-null  object 
 2   started_at          181806 non-null  object 
 3   ended_at            181806 non-null  object 
 4   start_station_name  152523 non-null  object 
 5   start_station_id    152523 non-null  object 
 6   end_station_name    150648 non-null  object 
 7   end_station_id      150648 non-null  object 
 8   start_lat           181806 non-null  float64
 9   start_lng           181806 non-null  float64
 10  end_lat             181678 non-null  float64
 11  end_lng             181678 non-null  float64
 12  member_casual       181806 non-null  object 
dtypes: float64(4), object(9)
memory usage: 18.0+ MB


# Merge data sources

In [6]:
# Merge datasets
data = pd.concat([oct2022, nov2022, dec2022], axis=0, ignore_index=True)

# Inspect dataset

In [7]:
# Display details of the resulting dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078226 entries, 0 to 1078225
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ride_id             1078226 non-null  object 
 1   rideable_type       1078226 non-null  object 
 2   started_at          1078226 non-null  object 
 3   ended_at            1078226 non-null  object 
 4   start_station_name  905631 non-null   object 
 5   start_station_id    905631 non-null   object 
 6   end_station_name    896192 non-null   object 
 7   end_station_id      896192 non-null   object 
 8   start_lat           1078226 non-null  float64
 9   start_lng           1078226 non-null  float64
 10  end_lat             1077393 non-null  float64
 11  end_lng             1077393 non-null  float64
 12  member_casual       1078226 non-null  object 
dtypes: float64(4), object(9)
memory usage: 106.9+ MB


In [8]:
# Inspect the first 10 rows
data.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A50255C1E17942AB,classic_bike,2022-10-14 17:13:30,2022-10-14 17:19:39,Noble St & Milwaukee Ave,13290,Larrabee St & Division St,KA1504000079,41.900680,-87.662600,41.903486,-87.643353,member
1,DB692A70BD2DD4E3,electric_bike,2022-10-01 16:29:26,2022-10-01 16:49:06,Damen Ave & Charleston St,13288,Damen Ave & Cullerton St,13089,41.920037,-87.677937,41.854967,-87.675700,casual
2,3C02727AAF60F873,electric_bike,2022-10-19 18:55:40,2022-10-19 19:03:30,Hoyne Ave & Balmoral Ave,655,Western Ave & Leland Ave,TA1307000140,41.979879,-87.681902,41.966400,-87.688704,member
3,47E653FDC2D99236,electric_bike,2022-10-31 07:52:36,2022-10-31 07:58:49,Rush St & Cedar St,KA1504000133,Orleans St & Chestnut St (NEXT Apts),620,41.902274,-87.627692,41.898203,-87.637536,member
4,8B5407BE535159BF,classic_bike,2022-10-13 18:41:03,2022-10-13 19:26:18,900 W Harrison St,13028,Adler Planetarium,13431,41.874754,-87.649807,41.866095,-87.607267,casual
5,A177C92E9F021B99,electric_bike,2022-10-13 15:53:27,2022-10-13 15:59:17,900 W Harrison St,13028,Loomis St & Lexington St,13332,41.874725,-87.649830,41.872187,-87.661501,casual
6,DF5EC7678DE3C2B3,electric_bike,2022-10-06 15:51:21,2022-10-06 15:55:06,900 W Harrison St,13028,Halsted St & Roosevelt Rd,TA1305000017,41.874713,-87.649875,41.867324,-87.648625,member
7,407DE6D80130A297,classic_bike,2022-10-26 17:30:10,2022-10-26 17:37:57,Rush St & Cedar St,KA1504000133,Larrabee St & Division St,KA1504000079,41.902309,-87.627691,41.903486,-87.643353,member
8,45EEAF68A1A051CA,classic_bike,2022-10-22 09:47:56,2022-10-22 09:57:42,Noble St & Milwaukee Ave,13290,Kingsbury St & Kinzie St,KA1503000043,41.900680,-87.662600,41.889177,-87.638506,casual
9,66CD8E4D0C38C0F3,electric_bike,2022-10-24 12:39:47,2022-10-24 12:48:36,Noble St & Milwaukee Ave,13290,Damen Ave & Cortland St,13133,41.900701,-87.662565,41.915983,-87.677335,member


# Clean dataset

## Remove null values

From the inspection of the dataset's metadata, it emerged there are missing values. Therefore we need to deal with them before proceeding in our analysis.

In [9]:
# Get the count of missing values per column
null_values_count = data.isnull().sum()

null_values_count

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    172595
start_station_id      172595
end_station_name      182034
end_station_id        182034
start_lat                  0
start_lng                  0
end_lat                  833
end_lng                  833
member_casual              0
dtype: int64

In [10]:
# Count the total number of cells inside the dataset
total_cells_count = np.product(data.shape)

# Count the total number of null values
total_null_values = null_values_count.sum()

# Compute the percentage of null values
null_values_percentage = (total_null_values/total_cells_count)*100

print(f"Null values represent the {null_values_percentage:.2f}% of the overall number of cells")

Null values represent the 5.07% of the overall number of cells


We have just discovered that null cells represent the `5.07%` of the overall cell count and the only fields where null values appear are those containing information about the start and end station names and ids, together with fields carrying information about latitude and longitude of the ride's end point.


Given that it is not much valuable for the purpose of the anlysis keeping rows where the information about the start or end points of the ride are missing, we can drop rows with null values.

In [11]:
# Remove rows with null values
data = data.dropna()

In [12]:
# Inspect again the count of null values per column
null_values_count = data.isnull().sum()

null_values_count

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

We have successfully got rid of null values. Let's now inspect once again the dataset's metadata to check the details of the subset of the initial dataset we are now left with.

In [13]:
# Display details of the clean dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 805466 entries, 0 to 1078225
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             805466 non-null  object 
 1   rideable_type       805466 non-null  object 
 2   started_at          805466 non-null  object 
 3   ended_at            805466 non-null  object 
 4   start_station_name  805466 non-null  object 
 5   start_station_id    805466 non-null  object 
 6   end_station_name    805466 non-null  object 
 7   end_station_id      805466 non-null  object 
 8   start_lat           805466 non-null  float64
 9   start_lng           805466 non-null  float64
 10  end_lat             805466 non-null  float64
 11  end_lng             805466 non-null  float64
 12  member_casual       805466 non-null  object 
dtypes: float64(4), object(9)
memory usage: 86.0+ MB


## Check uniqueness of ID values


Given that data comes from an external source, we cannot fully trust the way the information was gathered. As a consequence, we need to check if the ID fields present inside the dataset are actually unique.

If after inspecting these fields we discover some inconsistencies, additional cleaning steps might be necessary.

In [14]:
# Get the count of unique values per field
unique_values_per_col = data.nunique()

print(unique_values_per_col)

ride_id               805466
rideable_type              3
started_at            726294
ended_at              727791
start_station_name      1104
start_station_id        1067
end_station_name        1132
end_station_id          1091
start_lat             258576
start_lng             252549
end_lat                 1044
end_lng                 1036
member_casual              2
dtype: int64


### Check uniqueness of `ride_id` values

In [15]:
# Get the count of unique values for the ride_id column
unique_ride_ids = data['ride_id'].nunique()

print("The number of unique ride_ids is ",unique_ride_ids)

# Check if the unique count of ride_id values is coherent with the dataset size
if (unique_ride_ids) == len(data):
    print("Values of ride_ids are all unique")
else:
    print("Values of ride_ids are not unique")

The number of unique ride_ids is  805466
Values of ride_ids are all unique


### Check uniqueness of the mapping between `start_station_id` and `start_station_name`

In [16]:
# Get the count of unique values for the start_station_id column
unique_start_station_ids = data['start_station_id'].nunique()
print("The number of unique start_station_ids is ", unique_start_station_ids)

# Get the count of unique values for the start_station_name column
unique_start_station_names = data['start_station_name'].nunique()
print("The number of unique start_station_names is ", unique_start_station_names)

The number of unique start_station_ids is  1067
The number of unique start_station_names is  1104


It seems like there are more unique values for the `start_station_name` field than for the `start_station_id` one. It means that there are some stations with same id but with different names. In other words the mapping between names and ids is not one-to-one.

Let's further ivestigate to understand the nature of the problem.

In [17]:
# Isolate columns related to start stations
start_stations = data[['start_station_name','start_station_id']]

# Group by start_station_id to find the associated list of start_station_names
names_per_station_id = start_stations.groupby('start_station_id')['start_station_name'].agg(list).reset_index()

# Add a column with the number of unique start station names associated to each start_station_id
names_per_station_id['unique_names_count'] = names_per_station_id['start_station_name'].apply(lambda x: len(set(x)))

# Filter the rows associated to station_ids with multiple names
station_id_with_multiple_names = names_per_station_id[names_per_station_id['unique_names_count'] > 1]

# Print the number of station_ids with multiple names
print("Number of station_ids with multple names: ",len(station_id_with_multiple_names))


Number of station_ids with multple names:  37


Let's pick a sample row out of the ones that represent a `station_id` with multiple names in order to get a better understanding of the phenomenon.

In [18]:
# Choose a row index
idx = random.randint(0,len(station_id_with_multiple_names) - 1)

# Retrieve the corresponding row
sample_station = station_id_with_multiple_names.iloc[idx]

# Get the the station id of the selected row together with the corresponding station names
sample_station_id = sample_station['start_station_id']
station_names = sample_station['start_station_name']

# Print the names associated to the station id
for name in list(set(station_names)):
    print(f"Station ID: {sample_station_id}, Name station: {name}")

Station ID: 595, Name station: Wabash Ave & 87th St
Station ID: 595, Name station: Public Rack - Michigan Ave & 113th St


### Check uniqueness of the mapping between `end_station_id` and `end_station_name`

In [19]:
# Get the count of unique values for the end_station_id column
unique_end_station_ids = data['end_station_id'].nunique()
print("The number of unique end_station_ids is ",unique_end_station_ids)

# Get the count of unique values for the end_station_name column
unique_end_station_names = data['start_station_name'].nunique()
print("The number of unique end_station_names is ",unique_end_station_names)

The number of unique end_station_ids is  1091
The number of unique end_station_names is  1104


Just like what happened with start stations, it seems like there are more unique values for the `end_station_name` field than for the `end_station_id` one. It means that there are some stations with same id but with different names.In other words the mapping between names and ids is not one-to-one.

Let's further ivestigate to understand the entity of the problem.

In [20]:
# Isolate columns related to end stations
start_stations = data[['end_station_name','end_station_id']]

# Group by start_station_id to find the associated list of end_station_names
names_per_station_id = start_stations.groupby('end_station_id')['end_station_name'].agg(list).reset_index()

# Add a column with the number of unique end station names associated to each eend_station_id
names_per_station_id['unique_names_count'] = names_per_station_id['end_station_name'].apply(lambda x: len(set(x)))

# Filter the rows associated to station_ids with multiple names
station_id_with_multiple_names = names_per_station_id[names_per_station_id['unique_names_count'] > 1]

# Print the number of station_ids with multiple names
print("Number of station_ids with multple names: ",len(station_id_with_multiple_names))

Number of station_ids with multple names:  41


Let's pick a sample row out of the ones that represent a `station_id` with multiple names in order to get a better understanding of the phenomenon.

In [21]:
# Choose a row index
idx = random.randint(0, len(station_id_with_multiple_names) - 1)

# Retrieve the corresponding row
sample_station = station_id_with_multiple_names.iloc[idx]

# Get the the station id of the selected row together with the corresponding station names
sample_station_id = sample_station['end_station_id']
station_names = sample_station['end_station_name']

# Print the names associated to the station id
for name in list(set(station_names)):
    print(f"Station ID: {sample_station_id}, Name station: {name} ")

Station ID: 13074, Name station: Broadway & Wilson Ave 
Station ID: 13074, Name station: Broadway & Wilson - Truman College Vaccination Site 


### Check uniqueness of the mapping between `station_id` and `station_name`

Let's look at the problem from a broader perspective and let's analyze what happens when we do not distinguish between start or end stations.

In [22]:
# Create a view of station-related data
start_stations = data[['start_station_name','start_station_id']]
end_stations = data[['end_station_name','end_station_id']]

# Adjust field names of the Dataframes before mereging them
start_stations = start_stations.rename(columns={'start_station_name': 'station_name', 'start_station_id': 'station_id'})
end_stations = end_stations.rename(columns={'end_station_name': 'station_name', 'end_station_id': 'station_id'})

# Merge start and end station data
stations = pd.concat([start_stations, end_stations])

# Group by station_id to find the associated lists of station names
names_per_station_id = stations.groupby('station_id')['station_name'].agg(list).reset_index()

# Print the number of unique station_id fields
print("Number of unique station_id values:",len(names_per_station_id))

# Add a column with the number of unique station names associated with each station_id
names_per_station_id['unique_names_count'] = names_per_station_id['station_name'].apply(lambda x: len(set(x)))

# Filter the rows associated with station_ids having multiple names
station_ids_with_multiple_names = names_per_station_id[names_per_station_id['unique_names_count'] > 1]

Number of unique station_id values: 1120


In [23]:
# Print the number of station_ids with multiple names
print("Number of station_ids with multiple names: ", len(station_ids_with_multiple_names))

# Choose a row index
idx = random.randint(0, len(station_ids_with_multiple_names) - 1)

# Retrieve the corresponding row
sample_station = station_ids_with_multiple_names.iloc[idx]

# Get the station id of the selected row together with the corresponding station names
sample_station_id = sample_station['station_id']
station_names = sample_station['station_name']

# Print the names associated with the station id
for name in list(set(station_names)):
    print(f"Station ID: {sample_station_id}, Name station: {name} ")

Number of station_ids with multiple names:  47
Station ID: 546, Name station: Damen Ave & Pershing Rd 
Station ID: 546, Name station: Public Rack - Cicero Ave & Wellington Ave 


It turns out that the mapping problem affects a very limited portion of the overall dataset.

To address this issue multiple options are viable:

1. Solve the name clash by choosing a single name for each `station_id` by removing the rows with the discarded station names. However the name choice would be totally random, since no additional information is available on the way the mapping was originally done.
    
2. Drop the `station_id` columns, at the cost of keeping the discussed inconsistencies, which nonetheless affect only a small fraction of the dataset.

We will adopt the second strategy, since the station ids won't be used in the queries we plan to make later on in MongoDB. Indeed, station_name fields will be the ones on which queries will predicate on. Moreover query results will only include names for better readability. Therefore, we will fully embrace the schema-on-read philosophy of MongoDB and we will design the data model to include only the information required by queries.


In [24]:
data = data.drop(columns=['start_station_id','end_station_id'], axis=1)
data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
0,A50255C1E17942AB,classic_bike,2022-10-14 17:13:30,2022-10-14 17:19:39,Noble St & Milwaukee Ave,Larrabee St & Division St,41.900680,-87.662600,41.903486,-87.643353,member
1,DB692A70BD2DD4E3,electric_bike,2022-10-01 16:29:26,2022-10-01 16:49:06,Damen Ave & Charleston St,Damen Ave & Cullerton St,41.920037,-87.677937,41.854967,-87.675700,casual
2,3C02727AAF60F873,electric_bike,2022-10-19 18:55:40,2022-10-19 19:03:30,Hoyne Ave & Balmoral Ave,Western Ave & Leland Ave,41.979879,-87.681902,41.966400,-87.688704,member
3,47E653FDC2D99236,electric_bike,2022-10-31 07:52:36,2022-10-31 07:58:49,Rush St & Cedar St,Orleans St & Chestnut St (NEXT Apts),41.902274,-87.627692,41.898203,-87.637536,member
4,8B5407BE535159BF,classic_bike,2022-10-13 18:41:03,2022-10-13 19:26:18,900 W Harrison St,Adler Planetarium,41.874754,-87.649807,41.866095,-87.607267,casual


## Transfor and augment dataset

In [25]:
print(data['ride_id'].dtype)

# Rename the ride_id field
data = data.rename(columns={'ride_id':'_id'})

object


In [26]:
bike_types = data['rideable_type'].unique()

# Print the bike categories
print(bike_types)

# Print data type
print(data['rideable_type'].dtype)

# Rename the field
data = data.rename(columns={'rideable_type':'bike_type'})

# Drop the 'bike' string at the end of each bike_type value
data['bike_type'] = data['bike_type'].str.split('_').str[0]

# Change type to Categorical
data['bike_type'] = pd.Categorical(data['bike_type'])

bike_types = data['bike_type'].unique()

# Print the new bike categories
print(bike_types)

# Print the new data type
print(data['bike_type'].dtype)

['classic_bike' 'electric_bike' 'docked_bike']
object
['classic', 'electric', 'docked']
Categories (3, object): ['classic', 'docked', 'electric']
category


In [27]:
user_categories = data['member_casual'].unique()

# Print the user categories
print(user_categories)

# Rename the field
data = data.rename(columns={'member_casual':'user_category'})

# Print data type
print(data['user_category'].dtype)

# Change type to Categorical
data['user_category'] = pd.Categorical(data['user_category'])

# Print new data type
print(data['user_category'].dtype)

['member' 'casual']
object
category


In [28]:
# Print data type
print(data['started_at'].dtype)

# Change type to Datetime
data['started_at'] = pd.to_datetime(data['started_at'])

# Rename the field
data = data.rename(columns={'started_at':'start_time'})

# Print new data type
print(data['start_time'].dtype)

# Change type to Datetime
data['ended_at'] = pd.to_datetime(data['ended_at'])

# Rename the field
data = data.rename(columns={'ended_at':'end_time'})
print(data['end_time'].dtype)

# Add a new field to record the ride's length
data['ride_length'] = data['end_time'] - data['start_time']

# Print data type
print(data['ride_length'].dtype)

object
datetime64[ns]
datetime64[ns]
timedelta64[ns]


### Clean `ride_length` field

In [29]:
# Print dataset size
print(len(data))

# Remove rides of 0 length
rides_with_negative_len = data[data['ride_length'] < pd.Timedelta(seconds=0)]
data = data[data['ride_length'] > pd.Timedelta(seconds=0)]

# Print new dataset size
print(len(data))

# Print a subset of data points removed
rides_with_negative_len.head()

805466
805383


,_id,bike_type,start_time,end_time,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,user_category,ride_length
10284,F9A1F8F99C1EFBF5,electric,2022-10-03 08:55:01,2022-10-03 08:54:45,Chicago Ave & Sheridan Rd,Chicago Ave & Sheridan Rd,42.050523,-87.677829,42.050491,-87.677821,member,-1 days +23:59:44
154251,918F745F62CAC29E,classic,2022-10-13 14:42:10,2022-10-13 11:53:28,Wilton Ave & Diversey Pkwy*,Wilton Ave & Diversey Pkwy*,41.932418,-87.652705,41.932418,-87.652705,member,-1 days +21:11:18
572275,2F7E2E2160BD54FF,electric,2022-11-06 01:53:12,2022-11-06 01:30:03,Western Ave & Winnebago Ave,Campbell Ave & North Ave,41.915666,-87.687067,41.910535,-87.689556,casual,-1 days +23:36:51
574360,77417E7D945B4D32,electric,2022-11-06 01:54:17,2022-11-06 01:29:40,Western Ave & Winnebago Ave,Campbell Ave & North Ave,41.915603,-87.687143,41.910535,-87.689556,member,-1 days +23:35:23
588147,309BA8741F370C2B,electric,2022-11-06 01:39:08,2022-11-06 01:01:34,Sheridan Rd & Columbia Ave,Sheridan Rd & Columbia Ave,42.004624,-87.661533,42.004583,-87.661406,member,-1 days +23:22:26


In [30]:
# Inspect the maximum ride length
max_ride_len = data['ride_length'].max()
ride_with_max_len = data[data['ride_length'] == max_ride_len]
ride_with_max_len.head()

,_id,bike_type,start_time,end_time,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,user_category,ride_length
98297,B139FE7DF42819B0,docked,2022-10-24 14:37:54,2022-10-30 08:01:42,Field Museum,Bissell St & Armitage Ave*,41.865312,-87.617867,41.918296,-87.652183,casual,5 days 17:23:48


In [31]:
# Inspect the minimum ride length
min_ride_len = data['ride_length'].min()
ride_with_min_len = data[data['ride_length'] == min_ride_len]
ride_with_min_len.head()

,_id,bike_type,start_time,end_time,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,user_category,ride_length
12872,E42BA9D242CF9DA0,electric,2022-10-21 22:12:43,2022-10-21 22:12:44,Morgan St & 31st St,Morgan St & 31st St,41.837801,-87.651127,41.837800,-87.651140,member,0 days 00:00:01
17421,6A6427B4EF5C3EAE,electric,2022-10-02 18:37:37,2022-10-02 18:37:38,Honore St & Division St,Honore St & Division St,41.903136,-87.673773,41.903119,-87.673935,member,0 days 00:00:01
18222,2473D2B3EE45EBA5,classic,2022-10-10 12:37:35,2022-10-10 12:37:36,Halsted St & Polk St,Halsted St & Polk St,41.871840,-87.646640,41.871840,-87.646640,member,0 days 00:00:01
25039,F7323B884021D248,classic,2022-10-24 17:51:05,2022-10-24 17:51:06,Kosciuszko Park,Kosciuszko Park,41.930585,-87.723777,41.930585,-87.723777,member,0 days 00:00:01
25248,AC7462DD7B594090,electric,2022-10-24 17:53:48,2022-10-24 17:53:49,McClurg Ct & Ohio St,McClurg Ct & Ohio St,41.892574,-87.617059,41.892592,-87.617289,member,0 days 00:00:01


In [32]:
# Print data type
print(data['ride_length'].dtype)

# Transform ride length to milliseconds
data['ride_length'] = data['ride_length'].dt.total_seconds() * 1000
data['ride_length'] = data['ride_length'].astype(np.int32)
data = data.rename(columns={'ride_length':'ride_length_in_ms'})

print(data['ride_length_in_ms'].dtype)

timedelta64[ns]
int32


In [33]:
# Group start latitude and longitude fields inside an array of coordinates
data['start_coordinates'] = data.apply(lambda row: [row['start_lat'], row['start_lng']], axis=1)

# Drop individual latitude and longitude columns
data.drop(['start_lat', 'start_lng'], axis=1, inplace=True)

# Group end latitude and longitude fields inside an array
data['end_coordinates'] = data.apply(lambda row: [row['end_lat'], row['end_lng']], axis=1)

# Drop individual latitude and longitude columns of coordinates
data.drop(['end_lat', 'end_lng'], axis=1, inplace=True)

In [34]:
data.head()

,_id,bike_type,start_time,end_time,start_station_name,end_station_name,user_category,ride_length_in_ms,start_coordinates,end_coordinates
0,A50255C1E17942AB,classic,2022-10-14 17:13:30,2022-10-14 17:19:39,Noble St & Milwaukee Ave,Larrabee St & Division St,member,369000,"[41.90068, -87.6626]","[41.90348607004, -87.6433534936]"
1,DB692A70BD2DD4E3,electric,2022-10-01 16:29:26,2022-10-01 16:49:06,Damen Ave & Charleston St,Damen Ave & Cullerton St,casual,1180000,"[41.920037, -87.67793683333333]","[41.854966518753926, -87.67569959163664]"
2,3C02727AAF60F873,electric,2022-10-19 18:55:40,2022-10-19 19:03:30,Hoyne Ave & Balmoral Ave,Western Ave & Leland Ave,member,470000,"[41.979878902, -87.681902051]","[41.966399801840986, -87.68870428204536]"
3,47E653FDC2D99236,electric,2022-10-31 07:52:36,2022-10-31 07:58:49,Rush St & Cedar St,Orleans St & Chestnut St (NEXT Apts),member,373000,"[41.902273666666666, -87.6276925]","[41.898203, -87.637536]"
4,8B5407BE535159BF,classic,2022-10-13 18:41:03,2022-10-13 19:26:18,900 W Harrison St,Adler Planetarium,casual,2715000,"[41.874754, -87.649807]","[41.866095, -87.607267]"


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 805383 entries, 0 to 1078225
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   _id                 805383 non-null  object        
 1   bike_type           805383 non-null  category      
 2   start_time          805383 non-null  datetime64[ns]
 3   end_time            805383 non-null  datetime64[ns]
 4   start_station_name  805383 non-null  object        
 5   end_station_name    805383 non-null  object        
 6   user_category       805383 non-null  category      
 7   ride_length_in_ms   805383 non-null  int32         
 8   start_coordinates   805383 non-null  object        
 9   end_coordinates     805383 non-null  object        
dtypes: category(2), datetime64[ns](2), int32(1), object(5)
memory usage: 53.8+ MB


In [43]:
data.head().to_html('dataset.html')

# Save dataset

In [44]:
data.to_csv('data/bike_sharing.csv', index=False)